# v3_from_t3p2_ph2_hyper.ipynb

Hyperparameter search on the v3_from_t3p2_ph2 model using a genetic algorithm.

In [1]:
%cd ../..
import numpy as np
import matplotlib.pyplot as plt
import time
import pickle
from datetime import datetime
from multiprocessing import Pool

/home/z/projects/draculab


In [2]:
%load_ext Cython

In [3]:
%%cython
from draculab import *

In [4]:
%cd /home/z/projects/draculab/notebook/spinal/
from v3ft3p2ph2_net import *
%cd ../..

/home/z/projects/draculab/notebook/spinal
/home/z/projects/draculab


In [5]:
# A method to mutate individual parameters of a configuration
focus_params = False # whether to focus mutations on a specific set of parameters

# Specify paramters and ranges
# defaults are based on incumbent 9 of smac_pars in v3_normal_smac_test.ipynb
ranges = {"A__M_lrate" : {"low": 0.1, "high": 20., "default":4.5},
          "A__M_w_max_frac" : {"low": .05, "high": 1., "default":.33},
          "A__M_w_sum" : {"low": 1., "high": 40., "default":2.},
          "AL_thresh" : {"low": -.1, "high": 1., "default":.1},
          "b_e": {"low": .5, "high": 10., "default":3.},
          "C__C_antag": {"low": 0.1, "high": 3., "default":1.5},
          "C__C_p_antag": {"low": 0., "high": 1.5, "default":.5},
          "C__C_p_syne": {"low": 0., "high": 1., "default":.3},
          "C__C_syne": {"low": 0., "high": 2., "default":.5},
          "C_adapt_amp": {"low": 0., "high": 15., "default":8.},
          "C_cid" : {"low": 0.1, "high": 2., "default":.15},
          #"C_sigma" : {"low": 0., "high": 1., "default":.3}, # C_noise=False
          "C_slope" : {"low": 0.5, "high": 4., "default":2.1},
          "C_thresh" : {"low": -0.2, "high": 1.5, "default":.7},
          "CE__CI_w": {"low": 0., "high": 2.5, "default":1.},
          "CI__CE_w": {"low": -2.5, "high": 0, "default":-1.5},
          "g_e_factor": {"low": 0.5, "high": 4., "default":1.5},
          "M__C_lrate" : {"low": 5., "high": 500., "default":300.},
          "M__C_w_sum": {"low": 0.5, "high": 8., "default": 4.},
          "M__M_w": {"low": 0., "high": -3., "default":-1.},
          "M_cid": {"low": 0.05, "high": 2., "default": 1.},
          "M_des_out_w_abs_sum": {"low": 0.5, "high": 4., "default": 2.},
          "SPF__SPF_w": {"low": 0., "high": -3., "default":-.1}
         }


#par_list = [name for name in ranges] # ordered list with names of the parameters
par_list = list(ranges.keys())
# parameters to focus on
#main_pars = ["M_thresh", "M_slope", "M__C_lrate", "AF__M_w_sum", "AF__M_lrate", "M_p1_inp"]
if focus_params:
    par_list = main_pars

def mutate(cfg, name_list=par_list):
    """ Mutate a single parameter of the given configuration. 
    
        Args:
            name_list: list with names of candidate parameters.
    """
    n = np.random.randint(len(name_list))
    par_name = name_list[n]
    l = ranges[par_name]['low']
    h = ranges[par_name]['high']
    cfg[par_name] =  l + (h-l)*np.random.random()
    
def soft_mutate(cfg, ma, name_list=par_list):
    """ Soft-mutate a single parameter of the given configuration.
    
        A 'soft mutation' keeps the mutated value close to the original value.
        The maximum amplitude of the mutation is given by the 'ma' argument.
        
        Args:
            ma : float in (0,1]. Max. amplitude as fraction of the parameter's range 
            name_list: list with names of candidate parameters.
    """
    n = np.random.randint(len(name_list))
    par_name = name_list[n]
    l = ranges[par_name]['low']
    h = ranges[par_name]['high']
    cfg[par_name] = max(l, min(h, cfg[par_name] + ma * (h-l) * (np.random.random()-0.5)))

In [6]:
# Create the initial population: method 1

# create variations of the parameters we want to investigtate
pop_size = 60 # number of configurations in the population
pop = [{} for _ in range(pop_size)]
# first fill the default values in n_def elements
n_def = 3
for ind in range(n_def):
    for name in par_list:
        pop[ind][name] = ranges[name]['default']
        
# fill the rest with variations in both directions
chg_name = ["low", "default", "high"] # auxiliary list
for ind in range(n_def, pop_size):
    chg_dirs = [chg_name[i] for i in np.random.randint(3, size=len(par_list))]
    for idx, name in enumerate(par_list):
        pop[ind][name] = 0.5 * (ranges[name][chg_dirs[idx]] + ranges[name]["default"])
        
# Throw n_muts mutations, and n_soft_muts soft mutations
n_muts = 20
n_soft_muts = 10 # must have n_muts + n_soft_muts < pop_size-1
perm = np.random.permutation(range(1,pop_size)) # don't mutate the first element
for i in range(n_muts):
    mutate(pop[perm[i]])
for i in range(n_muts,n_muts+n_soft_muts):
    soft_mutate(pop[perm[i]], 0.2)

In [7]:
# reset fitness and number of evaluations
for dic in pop:
    dic['fitness'] = None # average fitness value
    dic['n_evals'] = 0  # number of times fitness has been evaluated
    
# Set search parameters present in the configurations
for cfg in pop:
    cfg['t_pres'] = 40
    cfg['par_heter'] = 0.01

In [8]:
# print used configuration
for dic in pop[0:3]:
    print('{',end='')
    for name in dic.keys():
        print("\'%s\':%s, "%(name, dic[name]), end='')
#         if name in net_conf:
#             print("\'%s\':%s, "%(name, dic[name]), end='')
#         if name != 'fitness' or dic['fitness'] != None:
#             print("\'%s\':%.2f, " % (name, dic[name]), end='')
    print('}\n')

pop[0]

{'A__M_lrate':4.5, 'A__M_w_max_frac':0.33, 'A__M_w_sum':2.0, 'AL_thresh':0.1, 'b_e':3.0, 'C__C_antag':1.5, 'C__C_p_antag':0.5, 'C__C_p_syne':0.3, 'C__C_syne':0.5, 'C_adapt_amp':8.0, 'C_cid':0.15, 'C_slope':2.1, 'C_thresh':0.7, 'CE__CI_w':1.0, 'CI__CE_w':-1.5, 'g_e_factor':1.5, 'M__C_lrate':300.0, 'M__C_w_sum':4.0, 'M__M_w':-1.0, 'M_cid':1.0, 'M_des_out_w_abs_sum':2.0, 'SPF__SPF_w':-0.1, 'fitness':None, 'n_evals':0, 't_pres':40, 'par_heter':0.01, }

{'A__M_lrate':4.5, 'A__M_w_max_frac':0.33, 'A__M_w_sum':2.0, 'AL_thresh':0.1, 'b_e':3.0, 'C__C_antag':1.5, 'C__C_p_antag':0.5, 'C__C_p_syne':0.3, 'C__C_syne':0.5, 'C_adapt_amp':8.0, 'C_cid':0.15, 'C_slope':2.1, 'C_thresh':0.30556045603148024, 'CE__CI_w':1.0, 'CI__CE_w':-1.5, 'g_e_factor':1.5, 'M__C_lrate':300.0, 'M__C_w_sum':4.0, 'M__M_w':-1.0, 'M_cid':1.0, 'M_des_out_w_abs_sum':2.0, 'SPF__SPF_w':-0.1, 'fitness':None, 'n_evals':0, 't_pres':40, 'par_heter':0.01, }

{'A__M_lrate':4.5, 'A__M_w_max_frac':0.33, 'A__M_w_sum':2.0, 'AL_thresh':0.1, 

{'A__M_lrate': 4.5,
 'A__M_w_max_frac': 0.33,
 'A__M_w_sum': 2.0,
 'AL_thresh': 0.1,
 'b_e': 3.0,
 'C__C_antag': 1.5,
 'C__C_p_antag': 0.5,
 'C__C_p_syne': 0.3,
 'C__C_syne': 0.5,
 'C_adapt_amp': 8.0,
 'C_cid': 0.15,
 'C_slope': 2.1,
 'C_thresh': 0.7,
 'CE__CI_w': 1.0,
 'CI__CE_w': -1.5,
 'g_e_factor': 1.5,
 'M__C_lrate': 300.0,
 'M__C_w_sum': 4.0,
 'M__M_w': -1.0,
 'M_cid': 1.0,
 'M_des_out_w_abs_sum': 2.0,
 'SPF__SPF_w': -0.1,
 'fitness': None,
 'n_evals': 0,
 't_pres': 40,
 'par_heter': 0.01}

In [9]:
# A function to produce offspring by crossing individuals
par_names = list(pop[0].keys()) # list with all parameter names

def create_offspring(cfg1, cfg2, par_list=par_names):
    """ Given 2 configurations, return 2 offspring from random swapping.
    
        To produce offspring, first we choose one split point in the
        dictionary. The first offspring has the values of cfg1 up to that
        point, and cfg2 afterwards. The second offspring has the cfg2 values
        up to the split point, and cfg1 afterwards. Since the dictionaries are
        not ordered, we use a parameter list to set the split point.
    
        Args:
            cfg1, cfg2: parameter dictionaries
            par_list: list with the keys in cfg1, cfg2
        Returns:
            cfg3, cfg4: dictionaries from swapping values in cfg1, cfg2
    """
    if focus_params:
        par_list = main_pars
    sp = np.random.randint(len(par_list))# split point as an index to par_list
    cfg3 = cfg1.copy()
    cfg4 = cfg2.copy()
    for i in range(sp, len(par_list)):
        cfg3[par_list[i]] = cfg2[par_list[i]]
        cfg4[par_list[i]] = cfg1[par_list[i]]
    return cfg3, cfg4

# visualize
# cfg3, cfg4 = create_offspring(pop[10], pop[11])
# for dic in (pop[10], pop[11], cfg3, cfg4):
#     print('{',end='')
#     for name in par_names:
#         if name != "fitness":
#             print("\'%s\':%.2f, " % (name, dic[name]), end='')
#     print('}\n')


In [10]:
# A function that evaluates the fitness of a given configuration
def eval_config(cfg):
    """ Returns the error for a network with a given configuration.

        Args:
            cfg : a configuration dictionary.
        Returns:
            error : A float calculated from the sum of activities in the SPF layer.
    """
    np.random.seed() # will try to get a seed from /dev/urandom
    if cfg['n_evals'] > 8: # if the fitness has been evaluated "enough" times. See cell below...
        return cfg['fitness']
    t_pres = cfg['t_pres']
    
    # obtain a network with the given configuration
    net, pops_dict, hand_coords, m_idxs = net_from_cfg(cfg,
               t_pres = cfg['t_pres'], 
               par_heter = cfg['par_heter'],
               set_C_delay = False,
               rand_targets = True,
               C_noise = False,
               track_weights = False,
               track_ips = False)
    # run the network
    run_time = 700.
    #start_time = time.time()
    times, data, plant_data  = net.flat_run(run_time)
    #print('Execution time is %s seconds' % (time.time() - start_time))

    # calculate average error in last half of reaching
    P = pops_dict['P']
    arm_activs = plant_data[P]
    plant = net.plants[P]
    # modified copy-paste of plt.upd_ip_impl
    q1 = arm_activs[:,0]
    q2 = arm_activs[:,2]
    q12 = q1+q2
    c_elbow = np.array((plant.l_arm*np.cos(q1), plant.l_arm*np.sin(q1)))
    c_hand = np.array((c_elbow[0] + plant.l_farm*np.cos(q12),
                    c_elbow[1] + plant.l_farm*np.sin(q12))).transpose()
    coord_idxs = np.floor(times/t_pres).astype(int)
    des_coords = np.array(hand_coords)[m_idxs[coord_idxs],:] # desired coordinates at each moment in time

    error_time = run_time - round(run_time/2.)
    error_idx = int(round(error_time/net.min_delay))
    hand_error = np.linalg.norm(c_hand-des_coords, axis=1)
    hand_error_integ = hand_error[error_idx:].sum()
    avg_hand_error = hand_error_integ / (hand_error.size - error_idx)

    #return hand_error_integ
    return avg_hand_error

In [ ]:
####################################
###### The genetic algorithm ######
####################################

# pop = pop[0:6] # limit pop size for debugging
n_mates = 10 # number of individuals to mate at each generation (even number)
max_gens = 80 # maximum number of generations
n_soft_mut = 6 # numbef of individuals to soft-mutate per generation
r_soft_mut = 0.2 # relative amplitude of soft mutations
n_mut = 6 # number of individuals to mutate per generation
n_procs = 30 # number of processes to use for fitness evaluation
n_save = 3 # number of individuals to protect from replacement and mutation
# setting name for file where parameters will be stored
fname = "v3ft3p2ph1_pop"
fname += "_" + datetime.now().strftime('%Y-%m-%d__%H_%M')

for gen in range(max_gens):
    start_time = time.time()
    # 1) Evaluate fitness
    # 1.1) Do the evaluation
    ######### Single process version
    #fits = list(map(eval_config, pop))
    ######## parallel version
    with Pool(n_procs) as p:
        fits = list(p.map(eval_config, pop))
        p.close()
        p.join()
    #print(fits)
    # 1.2) update the average fitness values
    for idx, cfg in enumerate(pop):
        nr = cfg['n_evals'] # n_evals is not updated yet...
        #print(nr)
        #print(fits[idx])
        #print(cfg['fitness'])
        if nr > 0:
            if nr < 9: # horrible arbitrary number that might be different in eval_config :( 
                cfg['fitness'] = (cfg['fitness']*nr + fits[idx])/(nr+1)
        else:
            cfg['fitness'] = fits[idx]
        cfg['n_evals'] = cfg['n_evals'] + 1
    # 2) Sort according to fitness. Lowest error first.
    pop = sorted(pop, key=lambda d: d['fitness'])
    # 2.1) Save current generation
    with open(fname, 'wb') as f:
        pickle.dump(pop, f)
        f.close()
    # 2.2) A quick message
    print("Generation %d evaluated. Best fitness: %.3f"%(gen,pop[0]['fitness']))
    print("Mean fitness = %.3f"%(np.mean(np.array(fits))))
    # 2.3) If best fitness good enough, break
    if pop[0]['fitness'] < 0.02:
        print("Good enough parameters found. Stopping search.")
        break
    # 3) mate and replace
    # 3.1) Select individuals to be replaced with probability proportional to error
    fits.sort() # sort the fitnesses (now in the same order as pop)
    fits = np.array(fits)
    fits = fits/fits.sum() # normalize fitnesses so they add to 1
    cumsum_fits = fits[:] # cumsum_fits[i] = sum(fits[:i])
    for i in range(1,len(cumsum_fits)):
        cumsum_fits[i] = cumsum_fits[i-1] + cumsum_fits[i]
    repl_list = [] # list with indexes of individuals to be replaced
    while len(repl_list) < n_mates:
        min_r = cumsum_fits[n_save] # don't replace the first n_save individuals
        r = min_r + (1.-min_r) * np.random.random()
        candidate = n_save
        for i in range(n_save, len(fits)):
            if cumsum_fits[i] > r:
                break
            candidate += 1
        if candidate in repl_list:
            continue
        else:
            repl_list.append(candidate)
    print("to replace: ", end='')
    print(repl_list)
    # 3.2) Arrange individuals in random pairs
    perm = np.random.permutation(n_mates) # this will do 
    # 3.3) mate
    new_pops = []
    for i in range(int(np.floor(n_mates/2))):
        off1, off2 = create_offspring(pop[perm[2*i]], pop[perm[2*i+1]])
        new_pops.append(off1)
        new_pops.append(off2)
    # 3.4) replace
    for i, cfg in enumerate(new_pops):
        pop[repl_list[i]] = cfg
    # 4) mutate
    # 4.1) soft mutations
    for _ in range(n_soft_mut):
        idx = np.random.randint(len(pop))
        if idx < n_save:
            copy = pop[idx].copy()
            soft_mutate(copy, r_soft_mut)
            pop[-idx-1] = copy
            pop[-idx-1]['fitness'] = None
            pop[-idx-1]['n_evals'] = 0
        else:
            soft_mutate(pop[idx], r_soft_mut)
            pop[idx]['fitness'] = None
            pop[idx]['n_evals'] = 0
    # 4.2) mutations
    # 4.2.1) select individuals to mutate
    # sq_fits = fits*fits
    #cumsum_sq_fits = fits * fits # cumsum_sq_fits[i] = sum(sq_fits[:i])
    cumsum_sq_fits = fits # proportional to fits, rather than its square
    cumsum_sq_fits = cumsum_sq_fits / cumsum_sq_fits.sum()
    for i in range(1,len(cumsum_sq_fits)):
        cumsum_sq_fits[i] = cumsum_sq_fits[i-1] + cumsum_sq_fits[i]
    mut_list = [] # list with indexes of individuals to be mutate
    while len(mut_list) < n_mut:
        r = np.random.random()
        candidate = 0
        for i in range(len(fits)):
            if cumsum_sq_fits[i] > r:
                break
            candidate += 1
        if candidate in mut_list:
            continue
        else:
            mut_list.append(candidate)
    print("to mutate: ", end='')
    print(mut_list)
    for idx in mut_list:
        if idx < n_save:
            copy = pop[idx].copy()
            mutate(copy)
            pop[-idx-1] = copy
            pop[-idx-1]['fitness'] = None
            pop[-idx-1]['n_evals'] = 0
        else:
            mutate(pop[idx])
            pop[idx]['fitness'] = None
            pop[idx]['n_evals'] = 0
            
    print('generation %d finished in %s seconds' % (gen, time.time() - start_time))
#     while len(mut_list) < n_mut:
#         min_r = cumsum_sq_fits[n_save] # don't mutate the first n_save individuals
#         r = min_r + (1.-min_r) * np.random.random()
#         candidate = n_save
#         for i in range(n_save, len(fits)):
#             if cumsum_sq_fits[i] > r:
#                 break
#             candidate += 1
#         if candidate in mut_list:
#             continue
#         else:
#             mut_list.append(candidate)
#     print("to mutate: ", end='')
#     print(mut_list)
#     for _ in range(n_mut):
#         idx = np.random.randint(len(pop))
#         mutate(pop[idx])
#         pop[idx]['fitness'] = None
#         pop[idx]['n_evals'] = 0
    

Generation 0 evaluated. Best fitness: 0.105
Mean fitness = 0.333
to replace: [30, 42, 17, 44, 50, 38, 43, 22, 59, 26]
to mutate: [35, 45, 57, 46, 54, 26]
generation 0 finished in 4523.554166555405 seconds
Generation 1 evaluated. Best fitness: 0.128
Mean fitness = 0.310
to replace: [45, 32, 14, 17, 53, 43, 21, 26, 42, 37]
to mutate: [9, 56, 8, 20, 47, 58]
generation 1 finished in 4455.522208213806 seconds
Generation 2 evaluated. Best fitness: 0.123
Mean fitness = 0.301
to replace: [18, 21, 47, 22, 24, 55, 41, 11, 49, 32]
to mutate: [56, 34, 23, 40, 35, 46]
generation 2 finished in 4453.975723028183 seconds
Generation 3 evaluated. Best fitness: 0.118
Mean fitness = 0.288
to replace: [46, 57, 18, 48, 40, 25, 53, 56, 47, 36]
to mutate: [55, 51, 41, 57, 56, 35]
generation 3 finished in 4777.422819375992 seconds
Generation 4 evaluated. Best fitness: 0.107
Mean fitness = 0.271
to replace: [58, 41, 42, 53, 47, 13, 57, 28, 21, 34]
to mutate: [57, 12, 58, 14, 44, 51]
generation 4 finished in 448

In [ ]:
# print final population
for dic in pop[:5]:
    print('{',end='')
    for name in dic.keys():
        if name != 'fitness' or dic['fitness'] != None:
            print("\'%s\':%.2f, " % (name, dic[name]), end='')
    print('}\n')

pop[0]

In [ ]:
# Save the final population
fname = "v3_nst_afx_pop"
fname += "_" + datetime.now().strftime('%Y-%m-%d__%H_%M')
with open(fname, 'wb') as f:
    pickle.dump(pop, f)
    f.close()

In [ ]:
# Load a saved population
import pickle
fname = 'v3_nst_afx_pop'
with (open(fname, "rb")) as f:
    results = pickle.load(f)
    f.close()

In [ ]:
# Test a configuation
cfg_id = 0 # index in the population for the configuration
net, pops_dict, hand_coords, m_idxs, t_pres, _  = net_from_cfg(pop[cfg_id])
pops_names = ['SF', 'SP', 'SPF', 'AL', 'AF', 'SP_CHG', 'CE', 'CI', 'M', 'ACT', 'P']
for name in pops_names:
    exec("%s = %s"% (name, str(pops_dict[name])))

start_time = time.time()
times, data, plant_data  = net.flat_run(600.)
#times, data, plant_data  = net.run(40.)
print('Execution time is %s seconds' % (time.time() - start_time))
data = np.array(data)
#Execution time is 8.687349319458008 seconds  << before sc_inp_sum_mp, flat_run(5.)

Same history as `v3_nst_afx`
...
